In [1]:
%pylab inline
import random
import re
import sys
import time
from multiprocessing import Process, Queue
import networkx as nx
import networkx.algorithms.centrality as nxcent
import networkx.algorithms.distance_measures as nxdist
import networkx.algorithms.components as nxcomp
import numpy as np
import elp_networks as elpnet
import elp_networks.algorithms as elpalg
import logbook

Populating the interactive namespace from numpy and matplotlib


In [2]:
net_file = "external/as20000102.csv"
exp_name = "simulate_router"

In [43]:
def rewire_butterfly(g, fraction):
    m = 7
    butterfly = elpnet.Butterfly(m)
    num_bnodes = m*2**m
    # Only sample nodes that can be completely rewired
    # This list maps butterfly node labels to router node labels
    rewire_nodes = random.sample([n for n in g.nodes() if len(g.neighbors(n)) >= 4], num_bnodes)
    random.shuffle(rewire_nodes)
    router_to_bf = dict([(r, b) for b, r in enumerate(rewire_nodes)])
    # Create list of possible edges that haven't been rewired yet.
    # Data is stored both as edge list and node->neighbors dict for easy sampling and updating.
    router_neighbors = {}
    router_edges = set()
    for v in rewire_nodes:
        # Only include edges with both ends in rewire set
        for w in g.neighbors(v):
            if w != v and w in rewire_nodes:
                try:
                    router_neighbors[v].add(w)
                except KeyError:
                    router_neighbors[v] = set([w])
                try:
                    router_neighbors[w].add(v)
                except KeyError:
                    router_neighbors[w] = set([v])
                router_edges.add( tuple(sorted([v,w])) )
    # Repeat for butterfly edges
    bfly_neighbors = {}
    bfly_edges = set()
    for bv in butterfly.int_nodes():
        for bw in butterfly.int_neighbors(bv):
            try:
                bfly_neighbors[bv].add(bw)
            except KeyError:
                bfly_neighbors[bv] = set([bw])
            try:
                bfly_neighbors[bw].add(bv)
            except KeyError:
                bfly_neighbors[bw] = set([bv])
            bfly_edges.add( tuple(sorted([bv,bw])) )
    # Calculate number of rewirings and perform them
    to_rewire = round(fraction * len(router_edges))
    complete = 0
    while complete < to_rewire:
        # Choose a remaining edge from the router network
        try:
            v, w = random.choice(list(router_edges))
        except IndexError:
            print "No valid edges remaining"
            print complete, to_rewire
        # Remove from graph and remaining edges
        g.remove_edge(v, w)
        router_edges.remove( (v,w) )
        router_neighbors[v].remove(w)
        router_neighbors[w].remove(v)
        # Find all possible rewirings
        bfly_rewirings = set()
        if v in rewire_nodes:
            # Find corresponding butterfly node and its remaining neighbors
            bv = router_to_bf[v]
            bfly_rewirings |= set([tuple(sorted([bv, bw])) for bw in bfly_neighbors[bv]])
        if w in rewire_nodes:
            bw = router_to_bf[w]
            bfly_rewirings |= set([tuple(sorted([bv, bw])) for bv in bfly_neighbors[bw]])
        # Choose one rewiring
        if len(bfly_rewirings) == 0:
            # Our chosen nodes have no corresponding butterfly edges left to rewire
            # Remove them from the list of options
            if v in rewire_nodes:
                for t in list(router_neighbors[v]):
                    if t == w:
                        continue # Edges and neighbors removed when chosen
                    router_edges.remove(tuple(sorted([v,t])))
                    router_neighbors[v].remove(t)
                    router_neighbors[t].remove(v)
            if w in rewire_nodes:
                for t in list(router_neighbors[w]):
                    if t == v:
                        continue # Edge and neighbors removed when chosen
                    router_edges.remove(tuple(sorted([w,t])))
                    router_neighbors[w].remove(t)
                    router_neighbors[t].remove(w)
            continue
        bv, bw = random.choice(list(bfly_rewirings))
        # Remove butterfly edge from remaining
        bfly_edges.remove( (bv, bw) )
        bfly_neighbors[bv].remove(bw)
        bfly_neighbors[bw].remove(bv)
        # Add rewired edge to graph
        new_v, new_w = rewire_nodes[bv], rewire_nodes[bw]
        g.add_edge(new_v, new_w)
        complete += 1

In [4]:
edge_list = []
whitespace = re.compile(r"\w+")
nodes = set()
with open(net_file, "rb") as f:
    for row in f:
        if row.startswith("#"):
            continue
        source, target = re.split(r"\W+", row.strip())
        source = int(source.strip())
        target = int(target.strip())
        nodes.add(source)
        nodes.add(target)
        edge_list.append( (source,target) )
node_count = len(nodes)

In [ ]:
def do_centrality_work(edge_list):
    g = nx.Graph(edge_list)
    centralities = nxcent.betweenness_centrality(g, normalized=False)
    v, c = max(centralities.iteritems(), key=lambda x: x[1])
    return v, c

def centrality_worker(edges_q, component_inq, centrality_outq):
    while True:
        edge_list = edges_q.get()
        component_inq.put(edge_list)
        v, c = do_centrality_work(edge_list)
        centrality_outq.put((v,c))
        if c > 0:
            next_edges = [(s,t) for s,t in edge_list if s != v and t != v]
            edges_q.put(next_edges)

In [ ]:
def do_component_work(edge_list):
    g = nx.Graph(edge_list)
    return list(nxcomp.connected_components(g))

def component_worker(component_inq, diameter_inq, size_inq):
    while True:
        edge_list = component_inq.get()
        components = do_component_work(edge_list)
        diameter_inq.put( (components, edge_list) )
        size_inq.put(components)

In [ ]:
def do_diameter_work(components, edge_list):
    giant_nodes = set(max(components, key=len))
    giant_edges = []
    for source, target in edge_list:
        if source in giant_nodes and target in giant_nodes:
            giant_edges.append( (source, target) )
    g = nx.Graph(giant_edges)
    return nxdist.diameter(g)

def diameter_worker(diameter_inq, diameter_outq):
    while True:
        components, edge_list = diameter_inq.get()
        diameter = do_diameter_work(components, edge_list)
        diameter_outq.put(diameter)

In [ ]:
def do_size_work(components):
    giant_nodes = max(components, key=len)
    total = sum([len(x) for x in components])
    try:
        result = float(total - len(giant_nodes)) / float(len(components) - 1)
    except ZeroDivisionError:
        result = 0
    return result

def size_worker(size_inq, size_outq):
    while True:
        components = size_inq.get()
        size_outq.put(do_size_work(components))

In [ ]:
edges_q = Queue()
centrality_outq = Queue()
component_inq = Queue()
diameter_inq = Queue()
diameter_outq = Queue()
size_inq = Queue()
size_outq = Queue()

edges_q.put(edge_list)

workers = []
workers.append(Process(target=centrality_worker, args=(edges_q, component_inq, centrality_outq)))
workers.append(Process(target=component_worker, args=(component_inq, diameter_inq, size_inq)))
workers.append(Process(target=diameter_worker, args=(diameter_inq, diameter_outq)))
workers.append(Process(target=size_worker, args=(size_inq, size_outq)))

for w in workers:
    w.daemon = True
    w.start()
    
exp = logbook.Experiment(exp_name)
log = exp.get_logger()
with open(exp.get_filename("targeted_router.csv"), "wb") as out:
    log.info("Starting")
    finished = 0
    out.write("removed,diameter,size,high_label,high_betweenness,node_count\n")
    while finished < node_count:
        log.info("Iteration {}".format(finished))
        log.info("  Finding betweenness")
        label, centrality = centrality_outq.get()
        log.info("  Finding diameter")
        diameter = diameter_outq.get()
        log.info("  Finding size")
        size = size_outq.get()
        log.info("  Writing row")
        row = [finished, diameter, size, label, centrality, node_count]
        out.write(",".join([str(d) for d in row]) + "\n")
        out.flush()
        finished += 1
        if centrality == 0:
            break
log.info("Finished successfully")


In [45]:
g = nx.Graph(edge_list)
rewire_butterfly(g, 0.5)

No valid edges remaining
1114 1140.0


NetworkXError: The edge 899-3650 is not in the graph

In [16]:
g.neighbors(4662)

[24, 4662, 2821, 6]

In [ ]:
list(elpnet.Butterfly(3).nodes())